In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import pandascharm as pc

from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62
from Bio.SubsMat.MatrixInfo import blosum45
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo

from sklearn.preprocessing import scale 
from sklearn import model_selection

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso


from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate
import scipy.stats as stats


In [ ]:
def removeoutlier_col(df,cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df[[cols]] < (Q1 - 1.5 * IQR)) |(df[[cols]] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out

In [ ]:
blosum62.update(((b,a),val) for (a,b),val in list(blosum62.items()))
blosum45.update(((b,a),val) for (a,b),val in list(blosum45.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag


In [ ]:
def encode(encoding, output, df_clean, aln, esm1b, key = None):
    

    ClustalAlign = AlignIO.read(aln, 'clustal')
    summary_align = AlignInfo.SummaryInfo(ClustalAlign )
    dframe = pc.from_bioalignment(ClustalAlign).transpose()
    sequences = dframe.loc[df_clean.index]
    
    y = df_clean['y_val']

    scaler = preprocessing.StandardScaler()
    
    
    if encoding == 'One-Hot-Encoder':

        one_hot = OneHotEncoder()
        encoded = one_hot.fit(sequences)
        X = encoded.transform(sequences).toarray()
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
           
    if encoding == 'Bag-of-Words':

        X = pd.DataFrame([ProteinAnalysis(i).count_amino_acids() for i in df_clean['Sequence']])
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    if encoding == 'bigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                st = i+j
                all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-1):
                try:
                    check = temp[k] + temp[k+1]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
    
    if encoding == 'trigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    st = i+j+k
                    all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-2):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
        
    if encoding == 'quadrogram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    for l in lst:
                        st = i+j+k+l
                        all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-3):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]+temp[k+3]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)

    if encoding == 'BLOSUM62':

        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum62, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
        
        
    if encoding == 'BLOSUM45':
        
        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum45, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)   
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
    if encoding == 'ESM1b':
        

        encoded = esm1b.loc[df_clean.index]
        X = np.array(encoded)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    return X, y, X_scaled, scaler


In [ ]:
def ml_process(encoding, output, df, aln, esm1b, temp=False):
    
    
    df_clean = removeoutlier_col(df,'Log'+output).copy()
    df_clean = df_clean.set_index('Index')
    

    X, y, X_scaled, scaler = encode(encoding, output, df_clean, aln, esm1b) 
    
    y_scaled = y

    X_scaled, y_scaled = shuffle(X_scaled, y_scaled, random_state=101)
    
    X, y = shuffle(X, y, random_state=101)
    

    lm = LinearRegression()   
    scores_lm = cross_validate(lm, X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lm.update({"Algorithm": "Linear regression", 'Sequence Representation Method': encoding})
                               
    lasso_reg=Lasso()
    scores_lasso = cross_validate(lasso_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lasso.update({"Algorithm": "LASSO", 'Sequence Representation Method': encoding})
                               
    rf = RandomForestRegressor(random_state=42)
    scores_rf = cross_validate(rf,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_rf.update({"Algorithm": "Random Forest", 'Sequence Representation Method': encoding})    
    
    tree_reg=DecisionTreeRegressor()
    scores_tree_reg = cross_validate(tree_reg,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Decision Tree", 'Sequence Representation Method': encoding})    
    
    svr_reg = SVR()
    scores_svr_reg = cross_validate(svr_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_svr_reg.update({"Algorithm": "SVR", 'Sequence Representation Method': encoding})    
        
    
    mlp_reg = MLPRegressor(random_state=101, max_iter=100)
    scores_mlp_reg = cross_validate(mlp_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_mlp_reg.update({"Algorithm": "Neural Network", 'Sequence Representation Method': encoding})    
    
    en = ElasticNet()
    scores_en = cross_validate(en, X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Elastic Network", 'Sequence Representation Method': encoding})    
    
    xgb_reg = xgb.XGBRegressor()
    scores_xgb_reg = cross_validate(xgb_reg,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_xgb_reg.update({"Algorithm": "XGBoost", 'Sequence Representation Method': encoding})    
    

   

    dfResults = pd.concat([pd.DataFrame(scores_lm), pd.DataFrame(scores_lasso), pd.DataFrame(scores_rf), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_svr_reg), pd.DataFrame(scores_mlp_reg), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_xgb_reg)])
        
    
    return dfResults


In [ ]:
enzyme = 'betaGlucosidasewithMutants'

df = pd.read_excel('betaGlucosidasewithMutantsOptimumTemperature.xlsx')

output = 'pNP-Glc kcat/Km (1/smM)'

methods = ['One-Hot-Encoder', 'Bag-of-Words', 'bigram', 'trigram', 'quadrogram', 'BLOSUM45', 'BLOSUM62', 'ESM1b']

aln = enzyme +'.aln'

x = datetime.datetime.now()
date = str(x.year)+str(x.month)+str(x.day)

df['y_val']= df['Temperature Optimum']
df['Log'+output]= np.log10(df[output])




In [ ]:
esm1b = pd.read_excel(enzyme+'ESM1b_embeddings.xlsx', index_col = 0)

In [ ]:
df_kcatKmTopt = df[df['Percentage Activity Depending on Optimum Temp']==1]

In [ ]:
summary=[]

for method in methods:
    dfR = ml_process(method, output , df_kcatKmTopt, aln, esm1b, temp = False)
    print(dfR)
    summary.append(dfR)



In [ ]:
result=pd.DataFrame()
for item in range(8):
    result=result.append(summary[item])
result.to_excel(date + 'Predicting Optimum Temperature' + enzyme +'5 CV.xlsx')

In [ ]:
df_res=result

In [ ]:
df_res['test_root_mean_squared_error']=df_res['test_neg_root_mean_squared_error'].abs()
df_res['test_mean_absolute_error']=df_res['test_neg_mean_absolute_error'].abs()

df_res['train_root_mean_squared_error']=df_res['train_neg_root_mean_squared_error'].abs()
df_res['train_mean_absolute_error']=df_res['train_neg_mean_absolute_error'].abs()

In [ ]:
test_res=df_res.groupby(['Algorithm', 'Sequence Representation Method'], as_index=False).agg({'test_r2':['mean','std'],
                                                                                             'test_root_mean_squared_error':['mean','std'],
                                                                                             'test_mean_absolute_error':['mean','std'],
                                                                                             'train_r2':['mean','std'],
                                                                                             'train_root_mean_squared_error':['mean','std'],
                                                                                             'train_mean_absolute_error':['mean','std']})

In [ ]:
test_res.to_excel(date + 'Predicting Optimum Temperature' + enzyme +'5 CV mean and std.xlsx')